In [1]:
from collections import OrderedDict
from typing import Any, Tuple, List, Dict, Iterable, Callable, Union

In [2]:
def _first_true(iterable: Iterable, predictor: Callable[..., bool]) -> Any:
    """Returns the first true value in the iterable."""
    return next(filter(predictor, iterable))

def _filter_transpose(source: List[Any], names: Iterable[str]) -> List[List[Any]]:
    return [list(map(lambda obj: getattr(obj, name), source)) for name in names]

def _set_is_unique(source: Iterable, sequence: Iterable, mask: Iterable = None) -> bool:
    try:
        if mask is None:
            _first_true(sequence, lambda item: item in source)
        else:
            masked_source: set = {k for k in source if k not in mask}
            _first_true(sequence, lambda item: item in masked_source)
        return False
    except StopIteration:
        return True

In [3]:
class SortedDict:
    """
    Mutable collection of unique item (that must implement an item.key), providing the functionality
      (1) Sequence-like fast indexable access; [i or slice]
      (2) Dictionary-like fast mappable access; [key of slice]
      (3) Collection remains ordered (invariant) based on each element's [item.key] value
      (4) Insert/remove (slow), append (fast if sorted), pop (fast)
      (5) Collection can only contain unique (w.r.t item.key) items


    --------------------------------------------------------------------------------------
    ----                        High Level Usage Documentation                        ----
    --------------------------------------------------------------------------------------
    series = SortedDict(iter)               creates collection using an iterable
    
    series = SortedDict.with_sorted(iter)   creates collection using a pre-sorted iterable
                                   - possible undefined behavior if iterable is not sorted

    ---------------------------------------------------------------
    --    Common Sequence/Mapping behavior                       --
    ---------------------------------------------------------------
    clear()         Removes all the items from the collection
    copy()          Return a shallow copy of the collection

    sc = sa + sb    Returns the concatenation of rhs collections
    sa += sb        Extends the collection with the contents of sb   
    sa == sb        Returns logical (equal) comparison of collection's items;
                        also implements analogous <, <=, !=, >=, and >

    ---------------------------------------------------------------
    --   Sequence-like behavior                                  --
    ---------------------------------------------------------------
    append(x)        Add an item to the end of the collection
    extend(t)        Extends collection with the contents of t
    insert(i, x)     Inserts x into collection at the index given by i 
    pop([i])         Returns the item at i and removes it from the collection; default i = -1
    sort()           Forces a sort of the collection; otherwise defered until the last possible moment


    ---------------------------------------------------------------
    --   Mapping-like behavior                                   --    
    ---------------------------------------------------------------
    get(key[, default])     Return the value for key if key is in the dictionary, else default; 
    items()                 Return an iterator of the collection's items ((key, value) pairs)
    keys()                  Returns an iterator of the collection's keys
    pop([key, default])     Returns the key and removes it from the collection, if present, else defualt; 
                                if no key, default is last element
    popitem()               Remove and return a (key, value) pair from collection; returned in LIFO order    
    setdefault(default)     If default.key is in collection, return value; otherwise insert/return default
    update([other])         Update the collection with the key/value pairs from other; overwriting keys
    values()                Returns an iterator of the collections values
    
    ** note type(key) is float  /// future update to generalize type ///

    ---------------------------------------------------------------
    --   Additional notes w.r.t. access using the [] operator    --
    ---------------------------------------------------------------
    sd[index] = value           Assignment will overwrite item @ index and sort if necessary;
                                    exeption if value.key nonunique and series[index].key != value.key
    sd[key] = value             Assignment will insert or overwrite existing item w/ equal key             

    sd[low : high : set] 
        (1) Works (nearly) identical to list if args are integers
        (2) Returns a consitant skice matching a closed interval if args are floats, set must be int 
        
    sd[str, ...]                Returns for each (outer list), a list along the key-axis;
                                    matching the args to members of collection elements,
                                    operation is approxamatly a filtered, transpose
        
    Finally, it is again noted that while the sorted property of SortedDict is invariant; 
    the sorting operation is defered until the last possible moment (e.g., access, index based assignment)
        
    """    
    _data: List[Any]
    _keys: Dict[float, int]
    _is_valid: bool
        
    def __add__(self, other: Iterable) -> 'SortedDict':
        self.__make_valid__()
        if not _set_is_unique(self._keys, {item.key for item in other}):
            raise ValueError(f'Nonunique key, location mismatch; identical keys found in collection') 
        self._is_valid = False
        return self.__class__(self._data + other)
    
    def __contains__(self, other: Iterable) -> bool:
        self.__make_valid__()
        return other in self._data
    
    def __eq__(self, other: Iterable) -> bool:
        self.__make_valid__()
        if isinstance(other, SortedDict):
            return self._data == other._data
        else:
            return self._data == SortedDict(other)._data   
        
    def __ge__(self, other: Iterable) -> bool:
        self.__make_valid__()
        if isinstance(other, SortedDict):
            return self._data >= other._data
        else:
            return self._data >= SortedDict(other)._data 
    
    def __getitem__(self, key: Union[int, float, slice, Iterable]) -> Union[List[Any], Any]:
        start: int
        stop: int        
        self.__make_valid__()
        
        # dictionary-like behavior
        if isinstance(key, float):
            return self.__class__.with_sorted([self._data[self._keys[key]]])
        
        # list-like behavior
        elif isinstance(key, int):
            return self.__class__.with_sorted([self._data[key]])
        
        # slicing behavior; both list and dict** like
        elif isinstance(key, slice):
            
            # dictionary-like behavior; if a dict were 'slice-able'
            if isinstance(key.start, float) or isinstance(key.stop, float):
                if key.start is None:
                    start = 0
                else:
                    start = _first_true(self._keys.items(), lambda x: key.start <= x[0])[1]
                if key.stop is None:
                    stop = len(self._data)
                else:
                    stop = _first_true(reversed(self._keys.items()), lambda x: x[0] <= key.stop)[1] + 1
                return self.__class__.with_sorted(self._data[start : stop : key.step])
            
            # list-like behavior
            else:
                return self.__class__.with_sorted(self._data[key])
        
        # transpose-like behavior; for each (list), return a list along key-axis
        elif isinstance(key, str):
            return _filter_transpose(self._data, [key])        
        elif hasattr(key, '__iter__'):
            return _filter_transpose(self._data, list(key))  
        
        else:
            raise TypeError(f'SeriesData indices must be integers, floats, slices, or iterable')
            
    def __gt__(self, other: Iterable) -> bool:
        self.__make_valid__()
        if isinstance(other, SortedDict):
            return self._data > other._data
        else:
            return self._data > SortedDict(other)._data             
            
    def __iadd__(self, other: Iterable) -> 'SortedDict':
        self.extend(other)
        return self            
        
    def __init__(self, iterable: Iterable) -> None:
        self._data = list(iterable)
        self._is_valid = False
        
    def __iter__(self):
        self.__make_valid__()
        yield from self._data

    def __le__(self, other: Iterable) -> bool:
        self.__make_valid__()
        if isinstance(other, SortedDict):
            return self._data <= other._data
        else:
            return self._data <= SortedDict(other)._data          
        
    def __len__(self):
        return len(self._data)
    
    def __lt__(self, other: Iterable) -> bool:
        self.__make_valid__()
        if isinstance(other, SortedDict):
            return self._data < other._data
        else:
            return self._data < SortedDict(other)._data    
        
    def __make_keys__(self) -> None:
        self._keys = OrderedDict([(item.key, i) for i, item in enumerate(self._data)])
        
    def __make_valid__(self, force: bool = False) -> None:
        if force or not self._is_valid:
            #print('-- making valid --')
            self._data.sort(key=lambda item: item.key)
            self.__make_keys__()
            self._is_valid = True 

    def __ne__(self, other: Iterable) -> bool:
        result = self.__eq__(other)
        if result is not NotImplemented:
            return not result
        return NotImplemented            
        
    def __repr__(self) -> str:
        self.__make_valid__()
        return 'SortedDict[' + ',\n '.join(item.__repr__() for item in self._data) + ']'  
    
    def __reversed__(self):
        self.__make_valid__()
        yield from self._data[::-1]
            
    def __setitem__(self, key: Union[int, float, slice, Iterable], value: Union[List[Any], Any]) -> None:
        start: int
        stop: int
        self.__make_valid__()
                
        # mappable-like behavior
        if isinstance(key, float):    # // future type generalization //
            if value.key in self._keys:
                self._data[self._keys[key]] = value
            else:
                self.append(value)                
        
        # sequence-like behavior
        elif isinstance(key, int):            
            if value.key in self._keys and key != self._keys[value.key]:
                raise ValueError(f'Nonunique key, location mismatch; identical key @ {self._keys[value.key]}')    
            else:
                self._data[key] = value
                self._is_valid = False
                
        elif isinstance(key, slice):
            
            # mappable-like behavior; ** if dict allowed slice-like replacement
            if isinstance(key.start, float):
                start = _first_true(self._keys.items(), lambda x: key.start <= x[0])[1]
                stop = _first_true(reversed(self._keys.items()), lambda x: x[0] <= key.stop)[1] + 1           
            
            # sequence-like behavior
            else:
                start = key.start
                stop = key.stop           
            
            masked: set = {i.key for i in self._data[:start]} | {i.key for i in self._data[stop:]}   
            if _set_is_unique(masked, {item.key for item in value}):
                self._data[start : stop] = value
                self._is_valid = False                
            else:
                raise ValueError(f'Nonunique key, location mismatch; identical keys found outside of slice')
        
        else:
            raise TypeError(f'SortedDict indices must be integers, floats, slices, or iterables')
        
    def __str__(self) -> str:
        self.__make_valid__()
        return '[' + ',\n '.join(item.__str__() for item in self._data) + ']'
    
    @classmethod
    def with_sorted(cls, iterable: Iterable):
        instance = cls(iterable)
        instance.__make_keys__()
        instance._is_valid = True
        return instance
    
    def append(self, value: Any) -> None:
        self.__make_valid__()
        if value.key in self._keys:
            raise ValueError(f'Nonunique key provided; {value.key} @ index {self._keys[value.key]}')             
        if len(self._data) > 0 and value.key < next(reversed(self._keys)):
            self._is_valid = False
        self._data.append(value)
        self._keys[value.key] = len(self._data) - 1
    
    def clear(self) -> None:
        self._data.clear()
        self._keys = OrderedDict({})
        self._is_valid = True
        
    def copy(self) -> None:
        self.__make_valid__()
        return self.__class__(self._data)
    
    def extend(self, iterable: Iterable):
        self.__make_valid__()
        if not _set_is_unique(self._keys, {item.key for item in iterable}):
            raise ValueError(f'Nonunique key, location mismatch; identical keys found in collection')            
        self._data.extend(iterable)
        self._is_valid = False
        
    def get(self, key: float, defualt: Any = None) -> Any:
        try:
            return self._data[self._keys[key]]
        except KeyError:
            return defualt
            
    def insert(self, key: int, value: Any) -> None:
        self[key] = value
            
    def items(self):
        yield from {item[0] : self._data[item[1]] for item in self._keys.items()}.items()
            
    def keys(self):
        yield from self._keys.keys()
        
    def pop(self, key: Union[int, float] = -1, default: float = None) -> Union[Any, float]:
        self.__make_valid__()
        if isinstance(key, int):
            self._is_valid = False
            return self._data.pop(key)
        if key in self._keys:
            self._is_valid = False
            self._data.pop(self._keys[key])
            return key 
        if default is not None:
            return default            
        raise KeyError(f'Key is not in collection and no default provided') 
        
    def popitem(self) -> Tuple[float, Any]:
        self.__make_valid__()
        if len(self._data) == 0:
            raise KeyError(f'Method popitem called on an empty collection') 
        return (self._keys.popitem()[0], self._data.pop())
    
    def setdefault(self, default: Any) -> Any:
        try:      
            self.append(default)
            return default
        except ValueError:
            return self._data[self._keys[default.key]]
    
    def sort(self) -> None:
        self.__make_valid__()
        
    def update(self, iterable: Iterable) -> None:
        for item in iterable:
            self[item.key] = item
        
    def values(self):
        self.__make_valid__()
        return self.__iter__()


In [4]:
from collections import namedtuple
from random import shuffle

Employee = namedtuple('Employee', ['name', 'id', 'key'])
setattr(Employee, '__str__', lambda x: f'Employee(name={x.name}, id={x.id})')
names = 'abcdefghijklmnopqrstuvwxyz'
keys = list(range(26))
shuffle(keys)
source = [Employee(name, i, float(key)) for i, (name, key) in enumerate(zip(names, keys))]

In [5]:
sd = SortedDict(source)
sd

SortedDict[Employee(name='f', id=5, key=0.0),
 Employee(name='w', id=22, key=1.0),
 Employee(name='h', id=7, key=2.0),
 Employee(name='a', id=0, key=3.0),
 Employee(name='q', id=16, key=4.0),
 Employee(name='c', id=2, key=5.0),
 Employee(name='x', id=23, key=6.0),
 Employee(name='z', id=25, key=7.0),
 Employee(name='b', id=1, key=8.0),
 Employee(name='o', id=14, key=9.0),
 Employee(name='n', id=13, key=10.0),
 Employee(name='p', id=15, key=11.0),
 Employee(name='y', id=24, key=12.0),
 Employee(name='i', id=8, key=13.0),
 Employee(name='m', id=12, key=14.0),
 Employee(name='e', id=4, key=15.0),
 Employee(name='k', id=10, key=16.0),
 Employee(name='r', id=17, key=17.0),
 Employee(name='d', id=3, key=18.0),
 Employee(name='t', id=19, key=19.0),
 Employee(name='l', id=11, key=20.0),
 Employee(name='g', id=6, key=21.0),
 Employee(name='s', id=18, key=22.0),
 Employee(name='u', id=20, key=23.0),
 Employee(name='j', id=9, key=24.0),
 Employee(name='v', id=21, key=25.0)]

In [6]:
print(sd[-10 :: 2])
print()
print(sd[20.0 : 24.9 : 2])
print()
print(sd[22:].__repr__())
print()
sd[10:26][:20.0:4]['name', 'id', 'key']

[Employee(name=k, id=10),
 Employee(name=d, id=3),
 Employee(name=l, id=11),
 Employee(name=s, id=18),
 Employee(name=j, id=9)]

[Employee(name=l, id=11),
 Employee(name=s, id=18),
 Employee(name=j, id=9)]

SortedDict[Employee(name='s', id=18, key=22.0),
 Employee(name='u', id=20, key=23.0),
 Employee(name='j', id=9, key=24.0),
 Employee(name='v', id=21, key=25.0)]



[['n', 'm', 'd'], [13, 12, 3], [10.0, 14.0, 18.0]]

In [7]:
sd.append(Employee('xx', 0, 26.00))
sd.append(Employee('xy', 0, 27.00))
sd.extend([Employee('xz', 0, 28.00)])
sd = sd + [Employee('xi', 0, 29.00)]
sd += [Employee('xj', 0, 30.00)]
sd.insert(13, Employee('xk', 0, 31.00))
print('done append')

sd[0] = Employee('0', -1, 0.0)
sd[0] = Employee('1', -1, 100.0)
print('done index')

sd[0:4] = [Employee(name, -2, 200 + float(key)) for i, (name, key) in enumerate(zip('2 3 4 5'.split(), range(4)))]
print('done slice')

sd[10.0 : 15.0] = [Employee(name, -3, 300 + float(key)) for i, (name, key) in enumerate(zip('6 7 8 9'.split(), range(4)))]
sd[20.0 : 21.5] = [Employee('10', -4, 50.0), Employee('11', -4, 51.0)]

done append
done index
done slice


In [8]:
sd

SortedDict[Employee(name='c', id=2, key=5.0),
 Employee(name='x', id=23, key=6.0),
 Employee(name='z', id=25, key=7.0),
 Employee(name='b', id=1, key=8.0),
 Employee(name='o', id=14, key=9.0),
 Employee(name='k', id=10, key=16.0),
 Employee(name='r', id=17, key=17.0),
 Employee(name='d', id=3, key=18.0),
 Employee(name='t', id=19, key=19.0),
 Employee(name='s', id=18, key=22.0),
 Employee(name='u', id=20, key=23.0),
 Employee(name='j', id=9, key=24.0),
 Employee(name='v', id=21, key=25.0),
 Employee(name='xx', id=0, key=26.0),
 Employee(name='xy', id=0, key=27.0),
 Employee(name='xz', id=0, key=28.0),
 Employee(name='xi', id=0, key=29.0),
 Employee(name='xj', id=0, key=30.0),
 Employee(name='xk', id=0, key=31.0),
 Employee(name='10', id=-4, key=50.0),
 Employee(name='11', id=-4, key=51.0),
 Employee(name='1', id=-1, key=100.0),
 Employee(name='2', id=-2, key=200.0),
 Employee(name='3', id=-2, key=201.0),
 Employee(name='4', id=-2, key=202.0),
 Employee(name='5', id=-2, key=203.0),
 Emp

In [9]:
itm = sd.popitem()
sd.pop()
sd.pop(-2)
sd.pop(301.0)
sd.setdefault(itm[1])
print(sd.setdefault(next(sd[-1].values())))

print()
sd.update([Employee(name, -3, 300 + float(key)) for i, (name, key) in enumerate(zip('6 7 8'.split(), range(3)))])

sc = sd.copy()
print(sc)

print()
print(sc == sc._data)
print(sc > sc._data)
print(sc >= sc._data)
print(sc < sc._data)
print(sc <= sc._data)

print()
sd.clear()
print(sd.__repr__())

print()
print(sc != sd)
sc

Employee(name=9, id=-3)

[Employee(name=c, id=2),
 Employee(name=x, id=23),
 Employee(name=z, id=25),
 Employee(name=b, id=1),
 Employee(name=o, id=14),
 Employee(name=k, id=10),
 Employee(name=r, id=17),
 Employee(name=d, id=3),
 Employee(name=t, id=19),
 Employee(name=s, id=18),
 Employee(name=u, id=20),
 Employee(name=j, id=9),
 Employee(name=v, id=21),
 Employee(name=xx, id=0),
 Employee(name=xy, id=0),
 Employee(name=xz, id=0),
 Employee(name=xi, id=0),
 Employee(name=xj, id=0),
 Employee(name=xk, id=0),
 Employee(name=10, id=-4),
 Employee(name=11, id=-4),
 Employee(name=1, id=-1),
 Employee(name=2, id=-2),
 Employee(name=3, id=-2),
 Employee(name=4, id=-2),
 Employee(name=5, id=-2),
 Employee(name=6, id=-3),
 Employee(name=7, id=-3),
 Employee(name=8, id=-3),
 Employee(name=9, id=-3)]

True
False
True
False
True

SortedDict[]

True


SortedDict[Employee(name='c', id=2, key=5.0),
 Employee(name='x', id=23, key=6.0),
 Employee(name='z', id=25, key=7.0),
 Employee(name='b', id=1, key=8.0),
 Employee(name='o', id=14, key=9.0),
 Employee(name='k', id=10, key=16.0),
 Employee(name='r', id=17, key=17.0),
 Employee(name='d', id=3, key=18.0),
 Employee(name='t', id=19, key=19.0),
 Employee(name='s', id=18, key=22.0),
 Employee(name='u', id=20, key=23.0),
 Employee(name='j', id=9, key=24.0),
 Employee(name='v', id=21, key=25.0),
 Employee(name='xx', id=0, key=26.0),
 Employee(name='xy', id=0, key=27.0),
 Employee(name='xz', id=0, key=28.0),
 Employee(name='xi', id=0, key=29.0),
 Employee(name='xj', id=0, key=30.0),
 Employee(name='xk', id=0, key=31.0),
 Employee(name='10', id=-4, key=50.0),
 Employee(name='11', id=-4, key=51.0),
 Employee(name='1', id=-1, key=100.0),
 Employee(name='2', id=-2, key=200.0),
 Employee(name='3', id=-2, key=201.0),
 Employee(name='4', id=-2, key=202.0),
 Employee(name='5', id=-2, key=203.0),
 Emp

In [10]:
sc_i = sc.items()
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))

print()
sc_i = sc.keys()
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))

print()
sc_i = sc.values()
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))

print()
sc_i = iter(sc)
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))

print()
sc_i = reversed(sc)
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))
print(next(sc_i))

print()
print(sc.get(303.0))
print(sc.get(304.0))

(5.0, Employee(name='c', id=2, key=5.0))
(6.0, Employee(name='x', id=23, key=6.0))
(7.0, Employee(name='z', id=25, key=7.0))
(8.0, Employee(name='b', id=1, key=8.0))

5.0
6.0
7.0
8.0

Employee(name=c, id=2)
Employee(name=x, id=23)
Employee(name=z, id=25)
Employee(name=b, id=1)

Employee(name=c, id=2)
Employee(name=x, id=23)
Employee(name=z, id=25)
Employee(name=b, id=1)

Employee(name=9, id=-3)
Employee(name=8, id=-3)
Employee(name=7, id=-3)
Employee(name=6, id=-3)

Employee(name=9, id=-3)
None
